In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import sklearn 
import seaborn as sns

writer_r = pd.ExcelWriter(r'./RF+SVM.xlsx')

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error,make_scorer
from sklearn.model_selection import GridSearchCV
data_path = r'./data.xlsx'
sheet_names= pd.ExcelFile(data_path).sheet_names

results = []

# Define RMSE
def rmse_scorer(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
rmse = make_scorer(rmse_scorer, greater_is_better=False)

for i in range(15):
    number = i
    sheet_name = sheet_names[number]
    print(sheet_name)

    data = pd.read_excel(data_path,skiprows=0,sheet_name=sheet_name )
    data= data.dropna()
    # print(data.columns)

    features=data[data.columns[2:]]
    labels=data[data.columns[1]]
# print(features, labels)

# Standardization
    scaler=preprocessing.StandardScaler().fit(features)
    features_=scaler.transform(features)
    features_=pd.DataFrame(features_,columns=features.columns)
    print(features_.shape)

# Shuffle data order
    indices = np.arange(features_.shape[0])
    np.random.shuffle(indices)
    features_ = features_.iloc[indices]
    labels = labels.iloc[indices]

# Read RF Important Features
    data_path_features = r'./RF_Feature_Ranking.xlsx'
    data_features = pd.read_excel(data_path_features,skiprows=0,sheet_name=sheet_name )
    data_features.columns = data_features.columns.astype(str)
    print(data_features.columns)
    important_features= data_features['1']
    print(important_features)
    
    number_features=[i+1 for i in range(len(important_features))]
    print(f'total length：{len(number_features)}')
    
# Define parameter value range    
    param_grid = {
    'C': np.arange(1,2.1,0.2),
    'gamma': np.arange(0.01,0.03,0.002)
    }
    
# Start yield prediction based on the top n important features
    for num in number_features:
        feature_name = [feature for feature in important_features[:num]]
        selected_X= features_[feature_name]
        SVM = SVR(kernel='rbf')
        scoring = {'R2': 'r2', 'RMSE': rmse}
        grid_search = GridSearchCV(estimator=SVM, param_grid=param_grid, cv=10, scoring=scoring, refit='R2', return_train_score=False)
        grid_search.fit(selected_X, labels)

# Print best parameters and results
        print("Best parameters:", grid_search.best_params_)
        print("Best cross-validation R2 score: {:.4f}".format(grid_search.cv_results_['mean_test_R2'].max()))
        print("Corresponding RMSE for best R2:", grid_search.cv_results_['mean_test_RMSE'][grid_search.cv_results_['mean_test_R2'].argmax()])

# Save parameters
        best_params = grid_search.best_params_
        best_r2_score = grid_search.cv_results_['mean_test_R2'].max()
        best_rmse_score = grid_search.cv_results_['mean_test_RMSE'][grid_search.cv_results_['mean_test_R2'].argmax()]
    
        results.append({
            'Feature number': num,
            'R2': best_r2_score,
            'RMSE': best_rmse_score,
            'C': best_params['C'],
            'Gamma': best_params['gamma']
        })
        print(f'For top {num} features, best R2: {best_r2_score:.4f}, RMSE: {best_rmse_score:.2f}')

    result_df = pd.DataFrame(results)
    print(result_df)
    result_df.to_excel(writer_r, sheet_name=sheet_name, index=False)

# Find the row corresponding to the highest R2 value.
    max_index = result_df['R2'].idxmax()
    best_row = result_df.loc[max_index]
    print(f"Best gamma for max R2: {best_row['Gamma']}, with feature count: {best_row['Feature number']}")
    

In [3]:
writer_r.save()
writer_r.close()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19992\1609063933.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer_r.save()
